In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model,Sequential
from keras.layers.advanced_activations import LeakyReLU

In [2]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.astype(np.float32)/127.5 -1

X_train = X_train.reshape(60000, 784)

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
from keras.optimizers import Adam

adam = Adam(lr = 0.0002, beta_1 = 0.5)

In [4]:
noise_dim = 10

generator = Sequential(name='generator')

generator.add(Dense(256, input_shape=(noise_dim,)))
generator.add(LeakyReLU())
generator.add(Dense(512))
generator.add(LeakyReLU())
generator.add(Dense(1024))
generator.add(LeakyReLU())
generator.add(Dense(784, activation='tanh'))

generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               2816      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 784)               80

In [5]:
from tensorflow.keras.initializers import RandomNormal

discriminator = Sequential(name='discriminator')

discriminator.add(Dense(1024, input_shape=(784,), kernel_regularizer=RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.5))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.5))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU())
discriminator.add(Dropout(0.5))
discriminator.add(Dense(1, activation='sigmoid'))

discriminator.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)             

In [6]:
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

In [7]:
discriminator.trainable =False

gan_input = Input(shape=(noise_dim,))
x = generator(gan_input)
output = discriminator(x)

In [8]:
gan = Model(gan_input, output, name='gan')
gan.summary()

Model: "gan"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
generator (Sequential)       (None, 784)               1463312   
_________________________________________________________________
discriminator (Sequential)   (None, 1)                 1460225   
Total params: 2,923,537
Trainable params: 1,463,312
Non-trainable params: 1,460,225
_________________________________________________________________


In [9]:
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [10]:
def get_batches(data, batch_size):
  batches=[]

  for i in range(data.shape[0]//batch_size):
    batch = data[i*batch_size:(i+1)*batch_size]
    batches.append(batch)
  return np.asarray(batches)

In [11]:
print(get_batches(X_train, 128).shape, get_batches(X_train, 128))

(468, 128, 784) [[[-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  ...
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]]

 [[-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  ...
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]]

 [[-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  ...
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]]

 ...

 [[-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  ...
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]]

 [[-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  ...
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -1. ... -1. -1. -1.]
  [-1. -1. -

In [12]:
def visualize_training(epoch, d_losses, g_losses):
  
  plt.figure(figzie=(8,4))
  plt.plot(d_losses, label='Discriminator Loss')
  plt.plot(g_losses, label='Generator Loss')
  plt.xlabel('Epoch')
  plt.ylabel('loss')
  plt.legend()
  plt.show()
  print('epoch: {}, Discriminator Loss: {}, Generator Loss: {}'.format(epoch, np.asarray(d_losses).mean(), np.asarray(g_losses).mean()))

  noise = np.random.normal(0,1,size=(24, noise_dim))
  generated_images = generator.predict(noise)
  generated_images = generated_images.reshape(-1, 28,28)

  plt.figure(figsize=(8,4))
  for i in range(generated_images.shape[0]):
    plt.subplot(4,6, i+1)
    plt.imshow(generated_images[i], interpolation='nearest', cmap='Greys_r')
    plt.axis('off')
  plt.tight_layout()
  plt.show()


In [13]:
%%time

EPOCHS = 100
BATCH_SIZE = 128

d_losses=[]
g_losses=[]

for epoch in range(1, EPOCHS+1):
  for real_images in get_batches(X_train, BATCH_SIZE):
    # print('real_images.shape :',real_images.shape, 'real_images:',real_images) - (128,784)
    
    input_noise = np.random.uniform(-1,1, size=[BATCH_SIZE, noise_dim])
    generated_images = generator.predict(input_noise)
    
    x_dis = np.concatenate([real_images, generated_images])
    #print('x_dis.shape:',x_dis.shape) -(256, 784)
    y_dis = np.zeros(2*BATCH_SIZE)
    y_dis[:BATCH_SIZE] = 1

    # discriminator 학습
    discriminator.trainable = True
    d_loss = discriminator.train_on_batch(x_dis, y_dis)

    # gan 학습
    noise = np.random.uniform(-1,1, size=[BATCH_SIZE, noise_dim])
    y_gan = np.ones(BATCH_SIZE)

    # discriminator 학습정지
    discriminator.trainable = False
    g_loss = gan.train_on_batch(noise, y_gan)

  d_losses.append(d_loss)
  g_losses.append(g_loss)

  if epoch ==1 or epoch % 5 ==0:
    visuaize_training(epoch, d_losses, g_losses)


TypeError: ignored